In [10]:
import math
import torch.nn as nn

In [11]:
def ScalingNet(lightingParameters,b,fmel,fblood,shading,specmask):
        #print("-----Scaling Net-----")
        #print("LightingParam shape: ",lightingParameters.shape)
        #print("b: ",b.shape)
        #print("fmel: ",fmel.shape)
        #print("fblood: ",fblood.shape)
        #print("shading shape: ",shading.shape)
        #print("specmask shape: ",specmask.shape)
        ####scaling####
        soft = nn.Softmax(dim=1)
        sig = nn.Sigmoid()
        
        numBatch = b.shape[0]
        lightWeights = soft(lightingParameters[:,0:15])
        weightA = lightWeights[:,0,]
        weightD = lightWeights[:,1,]
        Fweights = lightWeights[:,2:14]
        CCT =  lightingParameters[:,14]
        CCT = ((22-1) / (1 + torch.exp(-1*CCT))) + 1
        b = 6*sig(b) - 3
        Bgrid = b.reshape(2,1,1,numBatch)
        Bgrid = Bgrid/3
        fmel = 2*sig(fmel) - 1
        fblood = 2*sig(fmel) - 1
        shading = torch.exp(shading)
        specmask = torch.exp(shading)
        #print("weightA: ",weightA.shape)
        #print("weightD: ",weightD.shape)
        #print("CCT: ", CCT.shape)
        #print("Fweights: ", Fweights.shape)
        #print("b:", b.shape)
        #print("Bgrid: ", Bgrid.shape)
        #print("fmel: ",fmel.shape)
        #print("fblood: ",fblood.shape)
        #print("shading shape: ",shading.shape)
        #print("specmask shape: ",specmask.shape)
        
        return weightA,weightD,CCT,Fweights,b,Bgrid,fmel,fblood,shading,specmask

In [ ]:
def calc_illuminantD(CCT,illumDNorm):   
    #https://en.wikipedia.org/wiki/Standard_illuminant used the d series illuminant equation
    S0 = S1 = S2 = sD = np.zeros((CCT.shape[0],1,33))
    xD = yD = M = M1 = M2 = np.zeros((CCT.shape[0]))

    for x in range(CCT.shape[0]):
        for y in range (illumDNorm.shape[2]):
            for z in range (illumDNorm.shape[3]):
                S0[x,0,y] = illumDNorm[0,0,y,z] + S0[x,0,y]
       
    u,s,v = np.linalg.svd(illumDNorm[0,0,:,:])
    S1 = u[:,0]
    S2 = u[:,1]
    for i in range(CCT.shape[0]):
        T = CCT[i].detach().numpy()
        if 4000< T and T < 7000:
            xD[i] = 0.244063 + 0.09911*((1e3)/T) + 2.9678*((1e6)/np.power(T,2)) - 4.6070*((1e9)/np.power(T,3))
        else:
            xD[i] = 0.237040 + 0.24748*((1e3)/T) + 1.9018*((1e6)/np.power(T,2)) - 2.0064*((1e9)/np.power(T,3))
        
        yD[i] = -3.0*np.power(xD[i],2) +  2.870*xD[i] - 0.275
        M[i] = 0.0241 + 0.2562*xD[i] - 0.7341*yD[i]
        M1[i] = (-1.3515 - 1.7703*xD[i] + 5.9114*yD[i])/M[i]
        M2[i] = (.030 -31.4424*xD[i] + 30.0717*yD[i])/M[i]
        sD[i,:,:] = S0[i,:,:] + M1[i]*S1[i] + M2[i]*S2[i]
    
                                                        
    return torch.tensor(sD)